#Introduction
The **Aggregate_MultiPoints** script summarizes a set of point features allowing for these points to create an area and are used to calculate statistics. This results in a layer of displays of the number of points within each area.

---

The following libraries imported allow the code to run.
> * **arcpy** - Allows to run all train and test scripts in an interactive environment.
>* **env** - Accesses and uses the arcpy library to define variables in the env module.
>* **re** - Regular expression or re, uses a set of strings or searches for a specific pattern.
>* **Pandas** -  Used to model, analyze, and manipulate data sets.
>* **arcpy.sa** import * - From the arcpy.sa module, the user can use the **arcpy.sa import** to use spatial analysis.
>* **os** - Allows the creation and removal of directory folders, gathering data, and changing and finding the current directory. Users can also use **os** to interact with the operation systems.


In [ ]:
# Libraries used
import arcpy
from arcpy import env
import re
import pandas as pd
from arcpy.sa import *
import os

In [ ]:
# Define the parent folder and the geodatabase name
parent_folder = os.getcwd()
geodatabase_name = "example.gdb"

- Create file *geodatabase_path*. If the file exists, it will print for the user to access. However, if the file does not exist, a new file will be created for the path.

- The created workspace allows for variables to be placed on *geodatabase_path*. The *geodatabase_path* allows geoprocessing operations to process variables using arcpy as the default workspace. Set the parallel environment processing to = 0, where spatial analysis can be used. Multidimensional variables will not be used or false. The calculated results can be overwritten on an existing output file.

In [ ]:
# Create the file geodatabase
geodatabase_path = os.path.join(parent_folder, geodatabase_name)
if os.path.exists(geodatabase_path):
  print(geodatabase_path)
else:
  arcpy.CreateFileGDB_management(parent_folder, geodatabase_name)

In [ ]:
# Set the geodatabase as the workspace
arcpy.env.workspace = geodatabase_path

In [ ]:
# Set up the environment
arcpy.env.parallelProcessingFactor = 0
arcpy.CheckOutExtension("Spatial")
acrpy.env.mathMultidimensionalVariable = False
acrpy.env.overwriteOutput = True

- The user is asked to input the name of the data product and the point shape file that is used for the product.

- An example would be:
>*Enter the name of the data product:* **Cope**

  >*Enter the name of the point shape file, you want to use:* **Cope_Grid_30_Points3**

- These inputs are later used to create a file path, *gb_thirty_points*, and assigned the file to a variable called *table*. Using the *table* variable, a list is created with the field names containing fields that have the data product name. Next, a list is created named *fields*. The list *fields* is then used to create columns of a list of field names. Each column is created in a temporary list containing the column name and 'MEAN' which is appended to the temporary list under the *fields* list.

In [ ]:
# Set data product name
product = input("Enter the name of the data product: ")

# Select point file
point_shape_name = input('Enter the name of the point shape file you want to use: ')
gb_thirty_points = os.path.join(geodatabase_path, point_shape_name)

In [ ]:
table = gb_thirty_points

# List the fields you want to include
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry" and product in f.name]
print(columns)

In [ ]:
fields = []
for c in columns:
  temp = [c, 'MEAN']
  fields.append(temp)

- The user is asked to input the grid shape file path and name of the file. If it is not found, it will ask the user to input an acceptable name. The user will then input the output, **out_n**, to create a grid database name for the resolution.

- An example would be:
>*Enter the full file path and name of the grid shape file:* **E:\\share\\BIgRun\\Grids\\Grid_400.shp\n**

  >*Enter resolution of grid shape file:* **400**

  >*Enter name of output:* **Cope_400**

- Arcpy is used to convert the grid shape file to a feature class and aggregate points. These parameters include output name, polygon layer (grid database), and summary fields.

In [ ]:

grid_shape_file = input("Enter the full file path and name of the grid shape file: ")
while not os. path.isfile(grid_shape_file):
  print('The shapefile was not found.')
  co = input("Please try again: ")

# Set resolution
out_n = input('Enter name of output: ')
grid_db = 'Grid_'+ resolution
arcpy.conversion.FeatureClassToFeatureClass(grid_shape_file, geodatabase_path, grid_db)

arcpy.gapro.AggregatePoints(point_layer=gb_thirty_points, out_feature_class=out_n, polygon_or_bin='polygon', polygon_layer=grid_db, summary_fields=fields)

- Ask the user to input the output folder path, this is where the table will be saved. The arcpy will save the table as a CSV file and place the CSV file in the output folder.

- An example would be:
>*Enter output folder path for the table:* **E:\\share\\BIgRun\\Tables**

In [ ]:
out_folder = input('Enter output folder path for table: ')
arcpy.conversion.TableToTable(out_n, out_folder, out_n + '.csv')